<a href="https://colab.research.google.com/github/MlMauriciolopes/Faculdade-Impacta/blob/main/trabalho_final_webmining_prof_fernando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOME DOS PARTICIPANTES
### Nome : Maurício Lopes da Silva RA: 2102930

## Entendimento do negócio

## Entendimento dos dados

Utilizamos a base de dados do filme "Homem de ferro 3" para a primeira análise de dados e ,utilizando os modelos de dados do BeautifulSoup, conseguimos extrair o HTML das páginas e links.
Na implementação do Crawler, quando "não" usamos o Try, conseguimos extrair mais links automaticamente, porém, com algumas implementações, a célula dá erro, por isso, decidimos deixar o modelo original, contudo ele só busca um link.

In [9]:
# Dados do filme "Homem de ferro 3"
import requests 

init = 'https://www.imdb.com/title/tt1300854/reviews?ref_=tt_urv'
response = requests.get(init)
response

<Response [200]>

In [10]:
html = response.text 
html

'\n\n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>Iron Man Three (2013) - Iron Man Three (2013) - User Reviews - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n</script>\n<script>\n    if (typeof uex == \'function\') {\n      uex("ld", "LoadTitle",

In [ ]:
# Processamento do HTML
from bs4 import BeautifulSoup
bs = BeautifulSoup(html, 'html.parser')
bs

In [ ]:
# pegar/filtrar links
links = bs('a')
links

In [ ]:
# pegar/filtrar hrefs
hrefs = [l.get('href') for l in links]
hrefs

Implementação do Crawler

In [14]:
import requests
import os
# armezenar o conteúdo capturado
def salva_arquivo(nome, conteudo):
  pasta = '/content/drive/MyDrive/aula_web_mining/iron_man_3'
  n_files = len(os.listdir(pasta))+1
  with open(f'{pasta}/{nome}', 'w', encoding='utf-8') as file:
    file.write(conteudo)

# Adicionar links buscados(protocolo, caminho, recurso, porta)
from bs4 import BeautifulSoup
from requests.compat import urljoin
def busca_links(conteudo, link):
  bs = BeautifulSoup(conteudo, 'html.parser')
  conteudo = bs(id='styleguide-v2')[0]
  links = conteudo("a", attrs={'href':links_validos})
  links = [urljoin(link, l.get('href'))for l in links]
  return links

# Buscando links válidos (sem none, se não está vazio e, se não está com cerquilha)
def links_validos(href):
  return href and href[0] != "#" and href.startswith('/iron_man_3/') # PS: se der problema, só tirar o 'None'

# buscar link das páginas a partir de um link inicial
def crawler(buscar, buscados, stop=1):
  if buscar and len(buscados) < stop:
    link = buscar.pop()
    if link not in buscados:
      try:
        print('Baixando....', link)
        response = requests.get(link, timeout=10)
        bs = BeautifulSoup(response.text, "html.parser")
        conteudo = bs(id="styleguide-v2")[0].text
        titulo = bs("title")[0].text.replace('/', '-')
        salva_arquivo(f'iron_man_3_{len(buscados)+1}.html',conteudo)
        buscados.add(link)
        novos_links = busca_links(response.text, link)
        buscar.update(novos_links)
      except:
        pass
     # print(len(buscar))
    #crawler(buscar, buscados, stop)

init = 'https://www.imdb.com/title/tt1300854/reviews?ref_=tt_urv' # link inicial
crawler(buscar = set([init]),buscados = set([]), stop=100) # itens para buscar | stop = quantas páginas vai buscar

Baixando.... https://www.imdb.com/title/tt1300854/reviews?ref_=tt_urv


## Preparação dos dados

Na parte de reviews/ download de avaliações do Google Drive, algumas divs estavam dando problemas de versionamento, com isso, mantivemos a base de dados da Amazon.com.br, mas com outro produto. 
Importante: Mesmo com as implementações, também tive alguns problemas de extração da div, talvez por erro do site, mesmo implementando o nome correto das divs utilizando o inspecionar do navegador.

In [15]:
# Preparação textual dos dados
textos = bs("text")
for texto in textos:
 print(texto.text)

Crawler para obter informações

In [ ]:
import requests 
from bs4 import BeautifulSoup

# armezenar o conteúdo capturado
def salva_arquivo(nome, conteudo):
  pasta = '/content/drive/MyDrive/aula_web_mining/SM-G998B'
  with open(f'{pasta}/{nome}', 'w', encoding='utf-8') as file:
    file.write(conteudo)

import time, random
def crawler_kindle(base_url, i, n):
  if i <= n:
    print(i)
    response = requests.get(f'{base_url}{i}')
    bs = BeautifulSoup(response.text, "html.parser")
    reviews = bs(attrs={'data-hook':'review'})
    print("Reviews", len(reviews))

    for j, r in enumerate(reviews):
      title = r(attrs={'data-hook': 'review-title'})[0].text
      text = r(attrs={'data-hook': 'review-body'})[0].text
      stars = r(attrs={'data-hook': 'review-star-rating'})[0].text.split(',')[0].strip()
      salva_arquivo(f'{stars}/review_{i}_{j+1}', f'{title} {text}')

    i+=1
    time.sleep(random.randint(1,10))
    crawler_kindle(base_url, i, n)

# Análise Poco Pro 256GB / Amazon.com.br
base_url = 'https://www.amazon.com.br/Smartphone-Poco-PRO-256gb-8gb/product-reviews/B0919PLV7S/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
crawler_kindle(base_url,1,20)

## Modelagem

No teste de extração manual do HTML, utilizamos duas bases de sites de notícias, e implementamos o texto do dia, no caso o site do G1.com e o MSN.com, junto com as referências acima, mostrando as referências de palavras com suas frequências de repetição.

In [25]:
# Fonte: MSN.com
# link: https://www.msn.com/pt-br/entretenimento/noticias/m%C3%A3e-de-menino-autista-que-teve-celular-derrubado-por-cr7-recusa-encontro-nada-a-falar-com-ele/ar-AAW8lMz?ocid=winp1taskbar&cvid=0a430f04c5864c02a3d8cd323c1f1bdb
html1 = '<html>' \
'<head>' \
'Mãe de menino autista que teve celular derrubado por CR7 recusa encontro: Nada a falar com ele' \
'</head>' \
'<body>' \
'Sarah Kelly, mãe do menino autista de 14 anos que teve o celular derrubado por ' \
'Cristiano Ronaldo com um tapa, disse ter recusado convite para se encontrar ' \
'com o astro português após ser procurada pelos representantes do jogador. ' \
'Torcedores do Everton, ela e o filho, Jake Harding, foram convidados para' \
'conhecer o atacante no Old Trafford, estádio do Manchester United, mas não se interessaram.'\
'</body>' \
'</html>'

# Fonte: G1.com
# link: https://g1.globo.com/tecnologia/noticia/2022/04/13/por-que-o-whatsapp-web-tem-irritado-usuarios-com-sua-nova-versao.ghtml
html2 = '<html>' \
'<head>'\
'Por que o WhatsApp Web tem irritado usuários com sua nova versão'
'</head>'\
'<body>' \
'A nova versão do WhatsApp Web permite que os usuários acessem ' \
'mensagens mesmo quando não estão com o celular por perto. Mas a '\
'ferramenta nem sempre funciona da forma esperada, o que tem causado '\
'relatos furiosos na internet. '\
'</body>'\
'</html>'

'</head><body>A nova versão do WhatsApp Web permite que os usuários acessem mensagens mesmo quando não estão com o celular por perto. Mas a ferramenta nem sempre funciona da forma esperada, o que tem causado relatos furiosos na internet. </body></html>'

In [28]:
from bs4 import BeautifulSoup
texto1 = BeautifulSoup(html1, "html.parser").get_text(separator=' ')
texto2 = BeautifulSoup(html2, "html.parser").get_text(separator=' ')
print(texto1)
print(texto2)

Mãe de menino autista que teve celular derrubado por CR7 recusa encontro: Nada a falar com ele Sarah Kelly, mãe do menino autista de 14 anos que teve o celular derrubado por Cristiano Ronaldo com um tapa, disse ter recusado convite para se encontrar com o astro português após ser procurada pelos representantes do jogador. Torcedores do Everton, ela e o filho, Jake Harding, foram convidados paraconhecer o atacante no Old Trafford, estádio do Manchester United, mas não se interessaram.
Por que o WhatsApp Web tem irritado usuários com sua nova versão


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    binary=True,
    norm=None,
    use_idf=False
)
vetor = vectorizer.fit_transform([texto1, texto2])
print(vetor.toarray()) # textos que mostram relevânica/ calcular frequência binária
print(vectorizer.get_feature_names()) # mostra todas as features

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]]
['14', 'anos', 'após', 'astro', 'atacante', 'autista', 'celular', 'com', 'convidados', 'convite', 'cr7', 'cristiano', 'de', 'derrubado', 'disse', 'do', 'ela', 'ele', 'encontrar', 'encontro', 'estádio', 'everton', 'falar', 'filho', 'foram', 'harding', 'interessaram', 'irritado', 'jake', 'jogador', 'kelly', 'manchester', 'mas', 'menino', 'mãe', 'nada', 'no', 'nova', 'não', 'old', 'para', 'paraconhecer', 'pelos', 'por', 'português', 'procurada', 'que', 'recusa', 'recusado', 'representantes', 'ronaldo', 'sarah', 'se', 'ser', 'sua', 'tapa', 'tem', 'ter', 'teve', 'torcedores', 'traf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [30]:
# Frequência do termo 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    binary=False,
    norm='l1',
    use_idf=False
)
vetor = vectorizer.fit_transform([texto1, texto2])
print(vetor.toarray()) # textos que mostram relevânica/ calcular frequência binária
print(vectorizer.get_feature_names()) # mostra todas as features

[[0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.02702703
  0.02702703 0.04054054 0.01351351 0.01351351 0.01351351 0.01351351
  0.02702703 0.02702703 0.01351351 0.05405405 0.01351351 0.01351351
  0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
  0.01351351 0.01351351 0.01351351 0.         0.01351351 0.01351351
  0.01351351 0.01351351 0.01351351 0.02702703 0.02702703 0.01351351
  0.01351351 0.         0.01351351 0.01351351 0.01351351 0.01351351
  0.01351351 0.02702703 0.01351351 0.01351351 0.02702703 0.01351351
  0.01351351 0.01351351 0.01351351 0.01351351 0.02702703 0.01351351
  0.         0.01351351 0.         0.01351351 0.02702703 0.01351351
  0.01351351 0.01351351 0.01351351 0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.09090909 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [31]:
# TFIDF contagem dos termos
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    binary=False,
    norm='l1',
    use_idf=True
)
vetor = vectorizer.fit_transform([texto1, texto2])
print(vetor.toarray()) # textos que mostram relevânica/ calcular frequência binária
print(vectorizer.get_feature_names()) # mostra todas as features

[[0.01389264 0.01389264 0.01389264 0.01389264 0.01389264 0.02778528
  0.02778528 0.02965418 0.01389264 0.01389264 0.01389264 0.01389264
  0.02778528 0.02778528 0.01389264 0.05557056 0.01389264 0.01389264
  0.01389264 0.01389264 0.01389264 0.01389264 0.01389264 0.01389264
  0.01389264 0.01389264 0.01389264 0.         0.01389264 0.01389264
  0.01389264 0.01389264 0.01389264 0.02778528 0.02778528 0.01389264
  0.01389264 0.         0.01389264 0.01389264 0.01389264 0.01389264
  0.01389264 0.01976946 0.01389264 0.01389264 0.01976946 0.01389264
  0.01389264 0.01389264 0.01389264 0.01389264 0.02778528 0.01389264
  0.         0.01389264 0.         0.01389264 0.02778528 0.01389264
  0.01389264 0.01389264 0.01389264 0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.07020637 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
from nltk.tokenize.treebank import TreebankWordTokenizer
# Outros atributos de TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
stemmer = nltk.stem.RSLPStemmer()
tokenizer = nltk.tokenize.RegexpTokenizer('[^\W\d]{3,}')
def stemmer_tokenizer(doc):
  tokens = tokenizer.tokenize(doc)
  tokens = [stemmer.stem(t) for t in tokens]
  return tokens

r = stemmer_tokenizer("teste do tokenizer")
print(r)

vectorizer = TfidfVectorizer(
    binary=False,
    norm='l1',
    use_idf=True,
    
    stop_words=nltk.corpus.stopwords.words('portuguese'),
    tokenizer = stemmer_tokenizer
)
vetor = vectorizer.fit_transform([texto1, texto2])
print(vetor.toarray()) # textos que mostram relevânica/ calcular frequência binária
print(vectorizer.get_feature_names()) # mostra todas as features

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['test', 'tokeniz']
[[0.02       0.02       0.02       0.02       0.04       0.04
  0.02       0.02       0.02       0.04       0.02       0.04
  0.02       0.02       0.02       0.02       0.02       0.02
  0.         0.02       0.02       0.02       0.02       0.04
  0.04       0.02       0.         0.02       0.02       0.02
  0.02       0.02       0.02       0.04       0.02       0.02
  0.02       0.02       0.02       0.02       0.04       0.02
  0.02       0.02       0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.   

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aquel', 'aquil', 'del', 'entr', 'er', 'ess', 'est', 'estej', 'estev', 'estiv', 'estivér', 'estivéss', 'fom', 'form', 'foss', 'fôr', 'fôss', 'haj', 'hav', 'houv', 'houvér', 'houvéss', 'iss', 'ist', 'mesm', 'minh', 'muit', 'noss', 'nó', 'par', 'pel', 'qu', 'sej', 'ser', 'som', 'tenh', 'ter', 'tev', 'tinh', 'tiv', 'tivér', 'tivéss', 'tính', 'vo', 'voc', 'ér'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Avaliação

Na avaliação de sentimentos, utilizamos a base sugerida do US Airline Sentiment do Kaggle, que mostra o nível se satisfação dos clientes em relação as linhas aéreas norte americanas no ano de 2015, e alguns modelos, mostrou um pouco de insatisfação pelos comentários diante da tabela "airline_sentiment" onde, até um modelo chegou a 100% de precisão dos dados.

In [33]:
# Análise de sentimentos Twitter US Airline Sentiment 
# Analyze how travelers in February 2015 expressed their feelings on Twitter
# Fonte: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment?resource=download
import pandas as pd
dados = pd.read_csv('/content/drive/MyDrive/aula_web_mining/SM-G998B/archive/Tweets.csv')
dados.shape

(14640, 15)

In [49]:
dados.describe

<bound method NDFrame.describe of                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativere

In [ ]:
dados.head

In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer
# Outros atributos do TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
stemmer = nltk.stem.RSLPStemmer()
tokenizer = nltk.tokenize.RegexpTokenizer('[^\W\d]{3,}')

def stemmer_tokenizer(doc):
  tokens = tokenizer.tokenize(doc)
  tokens = [stemmer.stem(t) for t in tokens]
  return tokens

stopwords = [stemmer.stem(s) for s in nltk.corpus.stopwords.words('portuguese')]

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    binary=False,
    norm='l1',
    use_idf=False,
    stop_words = stopwords,
    tokenizer = stemmer_tokenizer
)

vetor = vectorizer.fit_transform(dados.airline_sentiment)
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))
print(vetor)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['es', 'estivés', 'fos', 'fôs', 'houvés', 'is', 'nos', 'tivés'] not in stop_words.
  % sorted(inconsistent)


['negativ', 'neutr', 'positiv']
3
  (0, 1)	1.0
  (1, 2)	1.0
  (2, 1)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 2)	1.0
  (7, 1)	1.0
  (8, 2)	1.0
  (9, 2)	1.0
  (10, 1)	1.0
  (11, 2)	1.0
  (12, 2)	1.0
  (13, 2)	1.0
  (14, 2)	1.0
  (15, 0)	1.0
  (16, 2)	1.0
  (17, 0)	1.0
  (18, 2)	1.0
  (19, 2)	1.0
  (20, 0)	1.0
  (21, 2)	1.0
  (22, 2)	1.0
  (23, 1)	1.0
  (24, 0)	1.0
  :	:
  (14615, 0)	1.0
  (14616, 0)	1.0
  (14617, 2)	1.0
  (14618, 0)	1.0
  (14619, 2)	1.0
  (14620, 0)	1.0
  (14621, 0)	1.0
  (14622, 0)	1.0
  (14623, 2)	1.0
  (14624, 0)	1.0
  (14625, 2)	1.0
  (14626, 0)	1.0
  (14627, 0)	1.0
  (14628, 2)	1.0
  (14629, 0)	1.0
  (14630, 2)	1.0
  (14631, 0)	1.0
  (14632, 1)	1.0
  (14633, 0)	1.0
  (14634, 0)	1.0
  (14635, 2)	1.0
  (14636, 0)	1.0
  (14637, 1)	1.0
  (14638, 0)	1.0
  (14639, 1)	1.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [55]:
list(vetor.toarray()[0])

[0.0, 1.0, 0.0]

In [56]:
from sklearn.model_selection import train_test_split
X = vetor
y = dados.airline_sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state=88)
#X_train.shape, X_test.shape
print(y_train.value_counts(), '\n', y_test.value_counts())

negative    6425
neutral     2169
positive    1654
Name: airline_sentiment, dtype: int64 
 negative    2753
neutral      930
positive     709
Name: airline_sentiment, dtype: int64


In [57]:
from sklearn import tree

dtree = tree.DecisionTreeClassifier(random_state=88)
dtree.fit(X_train, y_train)

y_predict = dtree.predict(X_test)
y_predict

array(['neutral', 'negative', 'negative', ..., 'positive', 'neutral',
       'neutral'], dtype=object)

In [58]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_predict)
acc

1.0

In [59]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(y_test, y_predict)
cf

array([[2753,    0,    0],
       [   0,  930,    0],
       [   0,    0,  709]])

In [60]:
from sklearn.metrics import precision_recall_fscore_support

prf = precision_recall_fscore_support(y_test, y_predict)
prf

(array([1., 1., 1.]),
 array([1., 1., 1.]),
 array([1., 1., 1.]),
 array([2753,  930,  709]))